In [1]:
from bs4 import BeautifulSoup

In [2]:
infile = open("data/kessler.XML","r")
contents = infile.read()
soup = BeautifulSoup(contents, 'lxml')

In [ ]:
soup

In [1]:
import requests
from selenium import webdriver
import json
import csv
from bs4 import BeautifulSoup
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
# driver=webdriver.Firefox()

In [ ]:
# NIVEL 1: WEBSCRAPPING AL LISTADO

no_pages=124   #automatizar?
dict_links={}
for page in range(no_pages):
    url = f'https://www.cnmv.es/portal/Consultas/MostrarListados.aspx?id=3&page={str(page)}'
#     driver.get(url)   con Selenium
    requests.get(url)  #con requests
    sopa_listado = BeautifulSoup(driver.page_source, 'html.parser')
    results=sopa_listado.find_all('li',{'id':"elementoPrimerNivel"})
 
    for result in results:
        short_link=result.find('a')['href']
        link='https://www.cnmv.es/portal/Consultas/'+str(short_link)
        descr=result.find('a')['title']
        dict_links[descr] = link
    
dict_links

In [71]:
#BACK-UP

import pandas as pd
import numpy as np

a=pd.DataFrame.from_dict(dict_links,orient='index', columns=['links'])
a.to_csv('data/dict_links.csv')

In [2]:
import pandas as pd
import numpy as np

a=pd.read_csv('data/dict_links.csv',index_col=0)
dict_links=a.to_dict(orient='dict')
dict_links=dict_links['links']

In [115]:
import requests
from selenium import webdriver
import json
import csv
from bs4 import BeautifulSoup
import os
import datetime
import re
import tqdm

# os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'

# download_dir='/home/alvaro/Dropbox/IRONHACK/final_project/data'
    
# fp = webdriver.FirefoxProfile()
# fp.set_preference("browser.download.folderList",2)
# fp.set_preference("browser.download.dir", download_dir)
# fp.set_preference("browser.download.manager.showWhenStarting", False)
# fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/xml;text/xbrl;application/xml;application/xbrl;text/html;application/octet-stream")
  


In [125]:
# NIVEL 2: WEBSCRAPPING AL RESULTADO/FONDO
# driver=webdriver.Firefox(firefox_profile=fp)
placeholder=[]
data={}
dates=[]

# IF FONDO.JASON exists pass

for fondo,link in dict_links.items():
    nif=link[-9:-1]
    if os.path.isfile(f'data/csv/{fondo}.csv'): 
        
#                 with open(f'data/{fondo}.csv', 'r') as f:
#                     lines=f.readlines()
#                     check=True
#                     for line in lines:
#                         if fondo in line and :
#                             check=False
#                     if check:
#                         xbrl = requests.get(xbrl_link)
#                         fondo_dict.update(xml_scrap(xbrl.content,nif))  
#                     else:
        print(fondo, 'skipped!')
        continue
    web_fondo=requests.get(link+'&vista=0') #con requests
    sopa_fondo = BeautifulSoup(web_fondo.content, 'html.parser')
    gestora=sopa_fondo.find('table',{'id':"ctl00_ContentPrincipal_gridGestora"}).find('a').text
    depos=sopa_fondo.find('table',{'id':"ctl00_ContentPrincipal_gridDepositaria"}).find('a').text

    web_fondo1=requests.get(link+'&vista=1') #con Requests
    sopa_fondo = BeautifulSoup(web_fondo1.content, 'html.parser')
    
    try:
        ISIN=sopa_fondo.find('td',{'data-th':"ISIN"}).find('a').text
    except:
        ISIN=''
    fecha_reg=sopa_fondo.find('td',{'data-th':"Fecha registro oficial"}).text
    fecha_reg=datetime.datetime.strptime(fecha_reg,'%d/%m/%Y')
    fecha_fin=sopa_fondo.find('td',{'data-th':"Fecha último folleto"}).text
    fecha_fin=datetime.datetime.strptime(fecha_fin,'%d/%m/%Y')   
    
    start=fecha_reg.year
    end=fecha_fin.year+1
    temp=False
    for i in range(start,end):
        url_year=link+f'&vista=1&fs=01%2f02%2f{i}'
        web_fondo_year=requests.get(url_year)   #con requests
        sopa_fondo = BeautifulSoup(web_fondo_year.content, 'html.parser')
        
        tables=sopa_fondo.find('div',{'id':"ctl00_ContentPrincipal_pnIPPS"})
        tds=tables.find_all('td', {'data-th':"Documentos"})
        for td in tds:
            pdf_link= 'https://www.cnmv.es/portal/Consultas/'+str(td.find_all('a')[0]['href'][3:]) 
#             print(pdf_link)
            xbrl_link= 'https://www.cnmv.es/portal/Consultas/'+str(td.find_all('a')[1]['href'][3:])  
#             print(xbrl_link)

            xbrl_ID=xbrl_link.split('{')[-1][:-1]  
            
#             fondo_dict['xbrl_ID']=xbrl_ID

            
            xbrl = requests.get(xbrl_link)
            fondo_info=xml_scrap_main(xbrl.content,nif,fondo)

            if type(temp)==bool:
                    temp=pd.DataFrame.from_dict(fondo_info[0],orient='index', columns=['0']).T
            elif len(fondo_info)>1:
                for i in range(1,len(fondo_info)):
                    temp=temp.append(other=fondo_info[i],ignore_index=True)        
            elif len(fondo_info)==1:
                    temp=temp.append(other=fondo_info[0],ignore_index=True)  

    temp.to_csv(f'data/csv/{fondo}.csv', sep='*',index=False)
    print(f'{fondo}.csv created!')

# FONDOS CON CLASES
# FONDOS COMPARTIMENTADOS
# FONDOS CON CLASES Y COMPARTIMENTADOS

1 KESSLER GLOBAL, FI.csv created!
A&G RENTA FIJA CORTO PLAZO, FI.csv created!
ABACO GLOBAL VALUE OPPORTUNITIES FI.csv created!
ABACO RENTA FIJA MIXTA GLOBAL, FI.csv created!
ABANCA GESTION, FI.csv created!
ABANCA RENTA FIJA FLEXIBLE, FI.csv created!


KeyboardInterrupt: 

In [122]:
# NIVEL  3: SCRAPPING AL ARCHIVO XML

def xml_scrap_main(xbrl,nif,fondo):
    xml_soup = BeautifulSoup(xbrl, 'lxml')
#     print('xml_soup created with xml_scrap')
#     print(xml_soup)

    clases=xml_soup.find_all('iic-com:denominacionclase') 
    if clases!=[]:
        fondo_dict_list=[]
        for i in range(len(clases)):
            fondo_dict_list.append(xml_scrap_clase(xml_soup,nif,fondo,i))
        return fondo_dict_list
    else:
        return [xml_scrap(xml_soup,nif,fondo)]
    
        
def xml_scrap(xml_soup,nif,fondo):
    name=xml_soup.find('xbrli:identifier').text   
#     periodo
    startdate=str(xml_soup.find('xbrli:startdate').text)
    enddate=str(xml_soup.find('xbrli:enddate').text)

    start_date = datetime.date.fromisoformat(startdate)
    end_date = datetime.date.fromisoformat(enddate)

    if end_date.month-start_date.month<4:
        period_type='Trimester'
        if start_date.month==1:
            period_number=1
        if start_date.month==7:
            period_number=3
    if end_date.month-start_date.month>4:
        period_type='Semester'
        if start_date.month==1:
            period_number=1
        if start_date.month==7:
            period_number=2 

    full_period=f'{period_type} {period_number} {start_date.year}'

    #datos generales
    registro_CNMV=int(xml_soup.find('iic-com:registrocnmv').text)
    email_gest=xml_soup.find('dgi-est-gen:communicationvalue').text
    rating_depos=xml_soup.find('iic-com:ratingdepositario').text
    riesgo=xml_soup.find('iic-com:perfilriesgo').text

    #"FIM_S22020_V-66247677_da"
    context_ref=f'FIM_{full_period[0]}{period_number}{start_date.year}_V-{nif}_da'
    context_ref2=f'FIM_{full_period[0]}{period_number}{start_date.year}_V-{nif}_ia'
    context_ref3=f'FIM_{full_period[0]}{period_number}{start_date.year}_V-{nif}_daq'
    context_ref_short='da'
    context_ref2_short='ia'
    context_ref3_short='daq'       
    context_ref_nodash=f'FIM_{period_type[0]}{period_number}{start_date.year}_V{nif}_da'
    context_ref2_nodash=f'FIM_{period_type[0]}{period_number}{start_date.year}_V{nif}_ia'
    context_ref3_nodash=f'FIM_{period_type[0]}{period_number}{start_date.year}_V{nif}_daq'

    context_ref_list=[context_ref,context_ref_short,context_ref_nodash]
    context_ref2_list=[context_ref2,context_ref2_short,context_ref2_nodash]
    context_ref3_list=[context_ref3,context_ref3_short,context_ref3_nodash]
#         da: periodo actual
#         ia: fin periodo?
#         daq: quarter actual

    rotacion=xml_soup.find('iic-com:indicerotacioncartera',{'contextref':context_ref_list})  
    rentab_avg=xml_soup.find('iic-com:rentabilidadmedialiquidez',{'contextref':context_ref_list})
    n_participaciones=xml_soup.find('iic-com-fon:numeroparticipaciones',{'contextref':context_ref2_list})
    n_participes=xml_soup.find('iic-com-fon:numeroparticipes',{'contextref':context_ref2_list})
    beneficio=xml_soup.find('iic-com-fon:beneficiobrutoparticipacion',{'contextref':context_ref_list})
    dividendos=xml_soup.find('iic-com:distribuyedividendos',{'contextref':context_ref_list})
    patrimonio=xml_soup.find('iic-com:patrimonio',{'contextref':context_ref2_list})
    valor_liq=xml_soup.find('iic-com:valorliquidativo',{'contextref':context_ref2_list})
    comision_gest_pat=xml_soup.find('iic-com:comisiongestioncobradapatrimonio',{'contextref':context_ref_list})
    comision_gest_res=xml_soup.find('iic-com:comisiongestioncobradaresultados',{'contextref':context_ref_list})
    comision_gest_total=xml_soup.find('iic-com-fon:comisiongestioncobrada',{'contextref':context_ref_list})
    comision_depos=xml_soup.find('iic-com-fon:comisiondepositariocobrada',{'contextref':context_ref_list})

    #solo he cogido la del ultimo trimestre, rentabilidad extremas omitidas
    rentab_IIC_trim=xml_soup.find('iic-com:rentabilidadiic',{'contextref':context_ref_list})
    if rentab_IIC_trim==None:
        rentab_IIC_trim=xml_soup.find('iic-com:rentabilidadiic',{'contextref':context_ref3_list})

    #volatilidad de valor liquidativo, resto omitidas
    volat_vl_trim=xml_soup.find('iic-com-fon:volatilidadvalorliquidativo',{'contextref':context_ref_list,})
    if volat_vl_trim==None:
        volat_vl_trim=xml_soup.find('iic-com-fon:volatilidadvalorliquidativo',{'contextref':context_ref3_list})

    ratio_gastos_trim=xml_soup.find('iic-com:ratiototalgastos',{'contextref':context_ref_list})
    if ratio_gastos_trim==None:
        ratio_gastos_trim=xml_soup.find('iic-com:ratiototalgastos',{'contextref':context_ref3_list})

    float_vars=[rotacion,rentab_avg, n_participaciones,beneficio,valor_liq,
                comision_gest_pat, comision_gest_res, comision_gest_total,
               comision_depos, rentab_IIC_trim, volat_vl_trim, ratio_gastos_trim]
    int_vars=[n_participes,patrimonio]

    for k in range(len(float_vars)):
        fvar=float_vars[k]
        if fvar!= None:
            float_vars[k]=float(fvar.text)
    rotacion,rentab_avg, n_participaciones,beneficio,valor_liq,comision_gest_pat, comision_gest_res, comision_gest_total,comision_depos, rentab_IIC_trim, volat_vl_trim, ratio_gastos_trim=float_vars

    for k in range(len(int_vars)):
        ivar=int_vars[k]
        if ivar!= None:
            int_vars[k]=int(ivar.text)   
    n_participes,patrimonio=int_vars
    
    try:dividendos=dividendos.text
    except:pass
    
    return {"fondo":fondo, "name":name,"nif":nif, "period":full_period,
           'start_date':start_date,'end_date':end_date,'registro_CNMV':registro_CNMV,
           'email_gest':email_gest,'rating_depos':rating_depos,'riesgo':riesgo,
           'rotacion':rotacion , 'rentab_avg':rentab_avg,
           'n_participaciones':n_participaciones, 'n_participes':n_participes,
           'beneficio':beneficio,'patrimonio':patrimonio,
           'dividendos':dividendos,'valor_liq':valor_liq,
           'comision_gest_pat':comision_gest_pat,'comision_gest_res':comision_gest_res,
           'comision_gest_total':comision_gest_total,'comision_depos':comision_depos,
           'rentab_IIC_trim':rentab_IIC_trim,
           'volat_vl_trim':volat_vl_trim,'ratio_gastos_trim':ratio_gastos_trim,
            
           }

In [123]:
def xml_scrap_clase(xml_soup,nif,fondo,i):
    class_name=xml_soup.find_all('iic-com:denominacionclase')[i].text
    #periodo
    startdate=str(xml_soup.find('xbrli:startdate').text)
    enddate=str(xml_soup.find('xbrli:enddate').text)

    start_date = datetime.date.fromisoformat(startdate)
    end_date = datetime.date.fromisoformat(enddate)

    if end_date.month-start_date.month<4:
        period_type='Trimester'
        if start_date.month==1:
            period_number=1
        if start_date.month==7:
            period_number=3
    if end_date.month-start_date.month>4:
        period_type='Semester'
        if start_date.month==1:
            period_number=1
        if start_date.month==7:
            period_number=2 

    full_period=f'{period_type} {period_number} {start_date.year}'
    #general
    registro_CNMV=int(xml_soup.find('iic-com:registrocnmv').text)
    email_gest=xml_soup.find('dgi-est-gen:communicationvalue').text
    rating_depos=xml_soup.find('iic-com:ratingdepositario').text
    riesgo=xml_soup.find('iic-com:perfilriesgo').text
       
    #"FIM_S22020_V-66247677_da"
    context_ref=f'FIM_{period_type[0]}{period_number}{start_date.year}_V-{nif}_da'
    context_ref2=f'FIM_{period_type[0]}{period_number}{start_date.year}_V-{nif}_ia'
    context_ref3=f'FIM_{period_type[0]}{period_number}{start_date.year}_V-{nif}_daq'
    context_ref_short='da'
    context_ref2_short='ia'
    context_ref3_short='daq'       
    context_ref_nodash=f'FIM_{period_type[0]}{period_number}{start_date.year}_V{nif}_da'
    context_ref2_nodash=f'FIM_{period_type[0]}{period_number}{start_date.year}_V{nif}_ia'
    context_ref3_nodash=f'FIM_{period_type[0]}{period_number}{start_date.year}_V{nif}_daq'

    context_ref_list=[context_ref,context_ref_short,context_ref_nodash]
    context_ref2_list=[context_ref2,context_ref2_short,context_ref2_nodash]
    context_ref3_list=[context_ref3,context_ref3_short,context_ref3_nodash]
#         da: periodo actual
#         ia: fin periodo?
#         daq: quarter actual
    rotacion=xml_soup.find('iic-com:indicerotacioncartera',{'contextref':context_ref_list})
    rentab_avg=xml_soup.find('iic-com:rentabilidadmedialiquidez',{'contextref':context_ref_list})

    try:rotacion=float(rotacion.text)
    except:pass
    try:rentab_avg=float(rentab_avg.text)
    except:pass
    
    n_participaciones=xml_soup.find_all('iic-com-fon:numeroparticipaciones',{'contextref':context_ref2_list})
    n_participes=xml_soup.find_all('iic-com-fon:numeroparticipes',{'contextref':context_ref2_list})
    beneficio=xml_soup.find_all('iic-com-fon:beneficiobrutoparticipacion',{'contextref':context_ref_list})    
    dividendos=xml_soup.find_all('iic-com:distribuyedividendos',{'contextref':context_ref_list})
    patrimonio=xml_soup.find_all('iic-com:patrimonio',{'contextref':context_ref2_list})
    valor_liq=xml_soup.find_all('iic-com:valorliquidativo',{'contextref':context_ref2_list})
    comision_gest_pat=xml_soup.find_all('iic-com:comisiongestioncobradapatrimonio',{'contextref':context_ref_list})
    comision_gest_res=xml_soup.find_all('iic-com:comisiongestioncobradaresultados',{'contextref':context_ref_list})
    comision_gest_total=xml_soup.find_all('iic-com-fon:comisiongestioncobrada',{'contextref':context_ref_list})
    comision_depos=xml_soup.find_all('iic-com-fon:comisiondepositariocobrada',{'contextref':context_ref_list})

    #solo he cogido la del ultimo trimestre, rentabilidad extremas omitidas
    rentab_IIC_trim=xml_soup.find_all('iic-com:rentabilidadiic',{'contextref':context_ref_list})
    if rentab_IIC_trim==None:
        rentab_IIC_trim=xml_soup.find_all('iic-com:rentabilidadiic',{'contextref':context_ref3_list})
        
    #volatilidad de valor liquidativo, resto omitidas
    volat_vl_trim=clase.find_all('iic-com-fon:volatilidadvalorliquidativo',{'contextref':context_ref_list,})
    if volat_vl_trim==None:
        volat_vl_trim=clase.find_all('iic-com-fon:volatilidadvalorliquidativo',{'contextref':context_ref3_list})

    
    ratio_gastos_trim=clase.find_all('iic-com:ratiototalgastos',{'contextref':context_ref_list})
    if ratio_gastos_trim==None:
        ratio_gastos_trim=clase.find_all('iic-com:ratiototalgastos',{'contextref':context_ref3_list})

    
    float_vars=[n_participaciones,beneficio,valor_liq,
                comision_gest_pat, comision_gest_res, comision_gest_total,
               comision_depos, rentab_IIC_trim, volat_vl_trim, ratio_gastos_trim]
    int_vars=[n_participes,patrimonio]

    for j in range(len(float_vars)):
        fvar=float_vars[j]
        if fvar!= None:
            try:float_vars[j]=float(fvar[i].text)
            except:pass
    n_participaciones,beneficio,valor_liq,comision_gest_pat, comision_gest_res, comision_gest_total,comision_depos, rentab_IIC_trim, volat_vl_trim, ratio_gastos_trim=float_vars

    for j in range(len(int_vars)):
        ivar=int_vars[j]
        if ivar!= None:
            try:int_vars[j]=int(ivar[i].text)  
            except:pass
    n_participes,patrimonio=int_vars
    
    try:dividendos=dividendos.text
    except:pass
    
    return {"fondo":fondo, "name":class_name,"nif":nif, "period":full_period,
           'start_date':start_date,'end_date':end_date,'registro_CNMV':registro_CNMV,
           'email_gest':email_gest,'rating_depos':rating_depos,'riesgo':riesgo,
           'rotacion':rotacion , 'rentab_avg':rentab_avg,
           'n_participaciones':n_participaciones, 'n_participes':n_participes,
           'beneficio':beneficio,'patrimonio':patrimonio,'dividendos':dividendos,'valor_liq':valor_liq,
           'comision_gest_pat':comision_gest_pat,'comision_gest_res':comision_gest_res,
           'comision_gest_total':comision_gest_total,'comision_depos':comision_depos,
           'rentab_IIC_trim':rentab_IIC_trim,
           'volat_vl_trim':volat_vl_trim,'ratio_gastos_trim':ratio_gastos_trim,
            
           }  
        
        
        